# CUDA exercises

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
import itertools
%matplotlib inline

from IPython.display import display

In [ ]:
def csv_parse(path, header=False, sep=";"):
    if header:
        df = pd.read_csv(path, sep=sep, names=header)
    else:
        df = pd.read_csv(path, sep=sep)
    #display(df.head())
    return df



In [ ]:
def calcGB_Stride(df):
    return 3*8* df["N"]/df["k"] /1e9

def calcGB_Offset(df):
    return 3*8* (df["N"]-df["k"])/1e9

def transform_to_ms(df, cols):
    return df[cols]*1e3


# Ex6.1 Warps

In [ ]:
df = csv_parse("data/ph_data.csv", header=False)
df["time_dot*8"] = df["time_dot"]*8
display(df)

In [ ]:
cols = df.columns
times_cols = list(filter(lambda x: x not in ["N", "k"], cols))

cols, times_cols

In [ ]:
cpu_cols = list(filter(lambda x: "cpu" in x, times_cols))
gpu_cols = list(filter(lambda x: "cpu" not in x, times_cols))

cpu_cols, gpu_cols

In [ ]:
gpu_ref = ["time_dot", "time_dot*8","time_cublas"]
shared_cols = ["time_shared"]
warp_cols = list(filter(lambda x: "warp" in x, times_cols))
warp_cols, gpu_ref

In [ ]:
labels = ["time_shared", "warp", "warp_adapt", "dot", "dot*8", "cublas", "cpu_ref"]
labels

In [ ]:
def print_styling():

    plt.rc('figure', figsize=(14,8)) 
    SMALL_SIZE = 15
    MEDIUM_SIZE = 18
    BIGGER_SIZE = 26

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes

    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
fig, ax = plt.subplots()

logx, logy = True, True
if (logx and logy):
    plot_format = "loglog"
if (logx and not logy):
    plot_format = "logx"
if (not logx and logy):
    plot_format = "logy"
else:
    plot_format = "linear"

print_styling();
x = "N"
df.plot(x=x, y=shared_cols+warp_cols, logx=logx, logy=logy, marker="o", linestyle="-", ax=ax);
df.plot(x=x, y=gpu_ref+["time_cpuref"], logx=logx, logy=logy, linestyle="--", ax=ax);



fig.suptitle("Vector analysis runtime comparison")
plt.tight_layout()
ax.set_ylabel("runtime [s]")

ax.legend(labels)

ax.grid()

fig.savefig("vec_analysis_allN.pdf", bbox_inches="tight", pad_inches=0.2)

In [ ]:
fig, axs = plt.subplots(ncols=2)

logx, logy = True, True
if (logx and logy):
    plot_format = "loglog"
if (logx and not logy):
    plot_format = "logx"
if (not logx and logy):
    plot_format = "logy"
else:
    plot_format = "linear"

print_styling();
x = "N"
Nmax = 1e5
df[df["N"] <= Nmax].plot(x=x, y=shared_cols+warp_cols, logx=logx, logy=logy, marker="o", linestyle="-", ax=axs[0]);
df[df["N"] <= Nmax].plot(x=x, y=gpu_ref+["time_cpuref"], logx=logx, logy=logy, linestyle="--", ax=axs[0]);

Nmin = 1e4
df[df["N"] >= Nmin].plot(x=x, y=shared_cols+warp_cols, logx=logx, logy=logy, marker="o", linestyle="-", ax=axs[1]);
df[df["N"] >= Nmin].plot(x=x, y=gpu_ref, logx=logx, logy=logy, linestyle="--", ax=axs[1]);


fig.suptitle("Vector analysis runtime comparison")
plt.tight_layout()
axs[0].set_ylabel("runtime [s]")

axs[0].legend(labels)
axs[1].legend(labels[:-1])

for ax in axs.ravel():
    #ax.legend(labels)
    ax.grid()

fig.savefig("vec_analysis_partiotionedN.pdf", bbox_inches="tight", pad_inches=0.2)